In [1]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('Missing data').getOrCreate()

23/10/20 01:08:40 WARN Utils: Your hostname, Sundeeps-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 11.38.254.137 instead (on interface en0)
23/10/20 01:08:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 01:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 01:08:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.csv('/Users/sundeepk/Desktop/Employees.csv', header = True, inferSchema = True)

In [3]:
df.show()

+------+----+-----------+------+
|  name| age|experience |salary|
+------+----+-----------+------+
|Krish |  31|         10|  3000|
| Sudha|  30|          8| 25000|
| sunny|  27|          3| 20000|
|  paul|  24|          1| 20000|
|Harsha|  21|          1| 15000|
|Shubam|  23|          2| 18000|
|mahesh|NULL|       NULL| 40000|
|  NULL|  34|         10|  NULL|
|  NULL|  23|       NULL|  NULL|
+------+----+-----------+------+



In [4]:
## Drops every row which has NULL

df.na.drop().show()

+------+---+-----------+------+
|  name|age|experience |salary|
+------+---+-----------+------+
|Krish | 31|         10|  3000|
| Sudha| 30|          8| 25000|
| sunny| 27|          3| 20000|
|  paul| 24|          1| 20000|
|Harsha| 21|          1| 15000|
|Shubam| 23|          2| 18000|
+------+---+-----------+------+



In [27]:
# Drops Null values only from that Subset

df.na.drop(how = 'all', subset= ['name']).show()

+------+----+-----------+------+
|  name| age|experience |salary|
+------+----+-----------+------+
|Krish |  31|         10|  3000|
| Sudha|  30|          8| 25000|
| sunny|  27|          3| 20000|
|  paul|  24|          1| 20000|
|Harsha|  21|          1| 15000|
|Shubam|  23|          2| 18000|
|mahesh|NULL|       NULL| 40000|
+------+----+-----------+------+



In [26]:
##Threshold 

#What is threshold =2 
#--> it means drop only if there are 2 non-NULL values in that row

df.na.drop(how = 'all', thresh = 3).show()

+------+---+-----------+------+
|  name|age|experience |salary|
+------+---+-----------+------+
|Krish | 31|         10|  3000|
| Sudha| 30|          8| 25000|
| sunny| 27|          3| 20000|
|  paul| 24|          1| 20000|
|Harsha| 21|          1| 15000|
|Shubam| 23|          2| 18000|
+------+---+-----------+------+



In [18]:
## Fill the missing values 

df.na.fill('Missing', ['age']).show()

+------+----+-----------+------+
|  name| age|experience |salary|
+------+----+-----------+------+
|Krish |  31|         10|  3000|
| Sudha|  30|          8| 25000|
| sunny|  27|          3| 20000|
|  paul|  24|          1| 20000|
|Harsha|  21|          1| 15000|
|Shubam|  23|          2| 18000|
|mahesh|NULL|       NULL| 40000|
|  NULL|  34|         10|  NULL|
|  NULL|  23|       NULL|  NULL|
+------+----+-----------+------+



In [19]:
df.show()

+------+----+-----------+------+
|  name| age|experience |salary|
+------+----+-----------+------+
|Krish |  31|         10|  3000|
| Sudha|  30|          8| 25000|
| sunny|  27|          3| 20000|
|  paul|  24|          1| 20000|
|Harsha|  21|          1| 15000|
|Shubam|  23|          2| 18000|
|mahesh|NULL|       NULL| 40000|
|  NULL|  34|         10|  NULL|
|  NULL|  23|       NULL|  NULL|
+------+----+-----------+------+



In [24]:
#mean of experience 
#LETS USE IMPUTER FUNCTION 

from pyspark.ml.feature import Imputer

imputer = Imputer(
        inputCols = ['age', 'experience ', 'salary'],
        outputCols = ["{}_imputed".format(c) for c in ['age', 'experience ', 'salary']]
        ).setStrategy("median")

In [25]:
imputer.fit(df).transform(df).show()

+------+----+-----------+------+-----------+-------------------+--------------+
|  name| age|experience |salary|age_imputed|experience _imputed|salary_imputed|
+------+----+-----------+------+-----------+-------------------+--------------+
|Krish |  31|         10|  3000|         31|                 10|          3000|
| Sudha|  30|          8| 25000|         30|                  8|         25000|
| sunny|  27|          3| 20000|         27|                  3|         20000|
|  paul|  24|          1| 20000|         24|                  1|         20000|
|Harsha|  21|          1| 15000|         21|                  1|         15000|
|Shubam|  23|          2| 18000|         23|                  2|         18000|
|mahesh|NULL|       NULL| 40000|         24|                  3|         40000|
|  NULL|  34|         10|  NULL|         34|                 10|         20000|
|  NULL|  23|       NULL|  NULL|         23|                  3|         20000|
+------+----+-----------+------+--------